# Vector Search with LanceDB

Search federal regulation comments using vector similarity and hybrid search (vector + BM25).

Connects to LanceDB on Cloudflare R2 and uses a local `sentence-transformers` model (`BAAI/bge-base-en-v1.5`) for query embeddings.

**Required env vars** (set in a `.env` file or export them):
- `R2_BUCKET_NAME` — R2 bucket name
- `R2_ACCESS_KEY_ID` / `R2_SECRET_ACCESS_KEY` — R2 API token credentials
- `R2_ENDPOINT` — e.g. `https://<account_id>.r2.cloudflarestorage.com`

In [ ]:
# !pip install lancedb sentence-transformers torch python-dotenv pandas

In [5]:
import os
import lancedb
import pandas as pd
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

load_dotenv("../frontend/.env.local")  # reuse frontend env if available
load_dotenv("../.env", override=False)

# Connect to LanceDB on R2
lance_uri = os.environ.get("LANCE_DB_URI") or f"s3://{os.environ['R2_BUCKET_NAME']}/lance-data"
db = lancedb.connect(lance_uri, storage_options={
    "aws_access_key_id": os.environ["R2_ACCESS_KEY_ID"],
    "aws_secret_access_key": os.environ["R2_SECRET_ACCESS_KEY"],
    "aws_endpoint": os.environ["R2_ENDPOINT"],
    "aws_region": "auto",
})
print("Tables:", db.list_tables())

table = db.open_table("comments")
print(f"Rows: {table.count_rows():,}")

# Load the same embedding model used during indexing (768d)
model = SentenceTransformer("BAAI/bge-base-en-v1.5")
print(f"Model loaded ({model.get_sentence_embedding_dimension()}d)")

Tables: tables=['comments'] page_token=None
Rows: 1,000


Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1734.92it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: BAAI/bge-base-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded (768d)


In [6]:
# Peek at a few rows
table.head(5).to_pandas().drop(columns=["vector"])

,id,text,docket_id,agency_code,title,comment,posted_date
0,FWS-R6-ES-2023-0216-11671,"Comment from Zukoski, Katie While I appreciate...",FWS-R6-ES-2023-0216,FWS,"Comment from Zukoski, Katie",While I appreciate that the Fish and Wildlife ...,2024-01-16T05:00:00Z
1,OSHA-2021-0007-96348,"Comment from Schulte, Mark; I am writing to op...",OSHA-2021-0007,OSHA,"Comment from Schulte, Mark;","I am writing to oppose rule OSHA-2021-0007, re...",2021-12-24T05:00:00Z
2,SSA-2025-0225-18054,"Comment from R., C. Social Security Administra...",SSA-2025-0225,SSA,"Comment from R., C.","Social Security Administration,To whom it may...",2025-12-12T05:00:00Z
3,HHS-OCR-2023-0013-0395,"Comment from Mueller, Elizabeth, HHS-OCR-2023-...",HHS-OCR-2023-0013,HHS,"Comment from Mueller, Elizabeth, HHS-OCR-2023-...",[SHARE YOUR STORY HERE!] We had an experience ...,2023-11-12T05:00:00Z
4,EPA-HQ-OAR-2005-0079-0671,Comment submitted by R. Murray [A-97-21-IV-G-3...,EPA-HQ-OAR-2005-0079,EPA,Comment submitted by R. Murray [A-97-21-IV-G-395],See attached,2024-01-12T05:00:00Z


## Vector Search

Encode a query with the local model and find nearest neighbors by cosine similarity.

In [13]:
def vector_search(query: str, limit: int = 10, agency: str | None = None) -> pd.DataFrame:
    """Search comments by vector similarity."""
    embedding = model.encode(query, normalize_embeddings=True)
    results = table.search(embedding).metric("cosine").limit(limit)
    if agency:
        results = results.where(f"agency_code = '{agency}'")
    return results.to_pandas().drop(columns=["vector"])

In [14]:
# Find comments about clean water regulations
import pandas as pd
pd.set_option('display.max_colwidth', None)

vector_search("clean water pollution regulations")

,id,text,docket_id,agency_code,title,comment,posted_date,_distance
0,COE-2025-0002-0241,Comment on FR Doc # 2025-11190 Association of California Water Agencies (ACWA) Comments on USACE NWPs (COE-2025-0002),COE-2025-0002,COE,Comment on FR Doc # 2025-11190,Association of California Water Agencies (ACWA) Comments on USACE NWPs (COE-2025-0002),2025-07-18T04:00:00Z,0.658900
1,CEQ-2025-0002-5612,"Comment from Skye, Carissa Please do not remove these protections. Our environment and drinking water are vital to our health!",CEQ-2025-0002,CEQ,"Comment from Skye, Carissa",Please do not remove these protections. Our environment and drinking water are vital to our health!,2025-03-05T05:00:00Z,0.702828
2,EPA-HQ-OW-2023-0222-0346,"Comment submitted by June Elliott-Cattell We need to save the Atlantic Sturgeon! On December 14th, 2023, the US EPA Region 3, announced proposed new water quality standards to ensure essential protection for aquatic life in the Delaware Estuary. The proposal includes a recognition that estuary reaches of the river are used for fish propagation and must be protected as such. Significantly, the proposal includes new dissolved oxygen criteria to ensure enough oxygen in the water to support the aquatic life of the River, including the most oxygen sensitive species, of which Atlantic Sturgeon is the MOST sensitive.",EPA-HQ-OW-2023-0222,EPA,Comment submitted by June Elliott-Cattell,"We need to save the Atlantic Sturgeon! On December 14th, 2023, the US EPA Region 3, announced proposed new water quality standards to ensure essential protection for aquatic life in the Delaware Estuary. The proposal includes a recognition that estuary reaches of the river are used for fish propagation and must be protected as such. Significantly, the proposal includes new dissolved oxygen criteria to ensure enough oxygen in the water to support the aquatic life of the River, including the most oxygen sensitive species, of which Atlantic Sturgeon is the MOST sensitive.",2024-02-26T05:00:00Z,0.707271
3,CDC-2023-0055-2874,Comment from Anonymous I support nationwide wastewater surveillance. Please make sure the public continues to have access to the data.,CDC-2023-0055,CDC,Comment from Anonymous,I support nationwide wastewater surveillance. Please make sure the public continues to have access to the data.,2023-09-11T04:00:00Z,0.728896
4,CEQ-2021-0002-12449,"Comment from Naples, Jean Dear Council on Environmental Quality,<br/><br/><span style='padding-left: 30px'></span>I am writing as a physician and public health advocate who strongly supports full protection for our environment. As you are awsare, for more than fifty years, the National Environmental Policy Act (NEPA) has given people the ability to comment upon federal projects that directly impact their communities. <br/><span style='padding-left: 30px'></span>At this time, I very much urge you to please strengthen the National Environmental Policy Act (NEPA) and undo the egregious damage caused by the Trump administration&rsquo;s actions that put the interests of polluters over the health and well-being of our communities. For half a century, communities have used NEPA to have a say in what projects are built in their neighborhoods and voice concerns about the consequences federal projects and polluting industries create. <br/><span style='padding-left: 30px'></span>I am writing in full support of the decision by the Biden administration to restore equity and certainty in federal decision-making under NEPA, This decision will protect the ability of the voices of communities fighting pollution who deserve a say in projects directly affecting their lives and livelihoods. I strongly urge the CEQ to please reverse the changes that allowed polluting industries to prepare their own environmental reviews because this policy diminishes the role of alternative options to particular projects. Additionally, I very much urgr the CEQ to please roll back the harmful provisions that silenced communities 

In [15]:
# Filter to EPA comments only
vector_search("air quality emissions standards", agency="EPA")

,id,text,docket_id,agency_code,title,comment,posted_date,_distance
0,EPA-HQ-OAR-2015-0072-4221,"Comment submitted by Brenda Bell The current set of standards for soot pollution are dangerously outdated, and are insufficient to protect the health of our environment and communities. EPA took an important step toward addressing the problem in your recent proposal, but we urge you to strengthen the level of protection in the final standards.<br/><br/>I am writing to express my support for stronger science-based standards soot emissions reaching levels no higher than 8 micrograms per cubic meter annually, and 25 micrograms per cubic meter daily to ensure cleaner air for everyone. Strengthening our current annual national soot standard of 12 micrograms per cubic meter to 8 micrograms per cubic meter would save 16,000 lives each year.<br/><br/>Exposure to, and inhalation of, soot is linked to a variety of heart and lung diseases, in addition to reduced lung development in children, higher rates of asthma, bronchitis, heart disease, cancer and early death.<br/><br/>Updating these decade old standards consistent with the health science will help protect our most vulnerable, including children, seniors and people with chronic illness. These groups are particularly susceptible to the impacts of soot inhalation and are more likely to suffer increased mortality rates, hospitalizations and visits to the ER.<br/><br/>Impacts like these are particularly felt in communities of color and low-income communities, who are more likely to live in areas co-located with sources of this deadly pollutant. But stronger federal standards could change this, shrinking disparities and ensuring cleaner air for everyone. <br/><br/>Please finalize the strongest possible soot standards soon. There is no time to waste.<br/><br/>Shalom,<br/><br/>I am a servant of Lord Yahweh(God) Almighty and the Most High Yahweh in heaven, Lord Jesus Christ who is Lord Yahwehs son and the Messiah or the Anointed One and the Holy Spirit. I have been given the authority to speak on their behalf. Lord Yahweh already knows about this issue. Lord Yahweh is waiting on his children to finally do something about this issue. Every day millions of Americans inhale air pollution from burning fossil fuels. This air pollution includes fine particle pollution a dangerous emission more commonly known as soot. Thank you for taking the time to read my comments.<br/><br/>Truly",EPA-HQ-OAR-2015-0072,EPA,Comment submitted by Brenda Bell,"The current set of standards for soot pollution are dangerously outdated, and are insufficient to protect the health of our environment and communities. EPA took an important step toward addressing the problem in your recent proposal, but we urge you to strengthen the level of protection in the final standards.<br/><br/>I am writing to express my support for stronger science-based standards soot emissions reaching levels no higher than 8 micrograms per cubic meter annually, and 25 micrograms per cubic meter daily to ensure cleaner air for everyone. Strengthening our current annual national soot standard of 12 micrograms per cubic meter to 8 micrograms per cubic meter would save 16,000 lives each year.<br/><br/>Exposure to, and inhalation of, soot is linked to a variety of heart and lung diseases, in addition to reduced lung development in children, higher rates of asthma, bronchitis, heart disease, cancer and early death.<br/><br/>Updating these decade old standards consistent with the health science will help protect our most vulnerable, including children, seniors and people with chronic illness. These groups are particularly susceptible to the impacts of soot inhalation and are more likely to suffer increased mortality rates, hospitalizations and visits to the ER.<br/><br/>Impacts like these are particularly felt in communities of color and low-income communities, who are more likely to live in areas co-located with sources of this deadly pollutant. But stronger 

## Full-Text Search (BM25)

Use the inverted index for keyword-based search.

In [16]:
def text_search(query: str, limit: int = 10) -> pd.DataFrame:
    """Search comments using BM25 full-text search."""
    return (
        table.search(query, query_type="fts")
        .limit(limit)
        .to_pandas()
        .drop(columns=["vector"])
    )

text_search("PFAS contamination drinking water")

,id,text,docket_id,agency_code,title,comment,posted_date,_score
0,EPA-HQ-OPPT-2023-0311-0136,"Comment submitted by Debbie Blair As EPA considers expanding the Safer Choice program, I request that it add the category of fertilizers or soil amendments. <br/><br/>Sometimes the choice of material drives the choice of system&mdash;as is the case for soil amendments. Chemical fertilizers&mdash;those granular, powdered, or liquid products identified by NPK (the percentages of nitrogen, phosphorus, and potassium)&mdash;consist of highly-soluble chemical salts that feed plants directly, but poison soil microorganisms. The choice of such fertilizers rules out an organic system in which fertility inputs feed organisms in the soil, which release substances that feed plants. Chemical fertilizers also contribute to climate change because they are produced with fossil fuels and limit the ability of soil to draw down (sequester) atmospheric carbon, thus undermining efforts to mitigate the climate emergency and the horrific fire and flooding events that destroy life. Chemical fertilizers, in addition, produce weak growth that invites insects to feed on the plants. Aphids, for example, have long been known to increase on plants fertilized with soluble nitrogen fertilizer. <br/><br/>EPA encourages the land application of biosolids (sewage sludge), noting benefits to soil fertility and structure and other &ldquo;economic and waste management benefits (e.g., conservation of landfill space; reduced demand on nonrenewable resources like phosphorus; and a reduced demand for synthetic fertilizers).&rdquo; However, as summarized by the Guardian, &ldquo;Now the practice is behind a growing number of public health problems. Spreading pollutant-filled biosolids on farmland is making people sick, contaminating drinking water and filling crops, livestock, and humans with everything from pharmaceuticals to PFAS.&rdquo; Biosolids used as fertilizer also contribute plastic to the soil. Unfortunately, many products derived from biosolids are sold as &ldquo;organic&rdquo; fertilizers. <br/><br/>The wise gardener or farmer chooses soil amendments that are not toxic to the soil, environment, or consumers. One way to ensure freedom from chemical fertilizers or biosolids is to look for the OMRI (Organic Materials Review Institute) seal. Organic standards specifically prohibit the use of sewage sludge (aka biosolids), but the home gardener may not always be able to discern the materials from which &ldquo;organic&rdquo; fertilizers are made. In addition, &ldquo;organic&rdquo; fertilizers sourced from composted manure generally come from nonorganic confined animal feeding operations (CAFOs) and may also contain undesirable contaminants such as antibiotics, antibiotic resistance genes, pesticide degradants, or heavy metals. Since these contaminants are not regulated, Safer Choice could offer an additional indicator of safety for gardeners and farmers. <br/><br/>Please add the category of fertilizers or soil amendments to Safer Choice. <br/><br/>Thank you.",EPA-HQ-OPPT-2023-0311,EPA,Comment submitted by Debbie Blair,"As EPA considers expanding the Safer Choice program, I request that it add the category of fertilizers or soil amendments. <br/><br/>Sometimes the choice of material drives the choice of system&mdash;as is the case for soil amendments. Chemical fertilizers&mdash;those granular, powdered, or liquid products identified by NPK (the percentages of nitrogen, phosphorus, and potassium)&mdash;consist of highly-soluble chemical salts that feed plants directly, but poison soil microorganisms. The choice of such fertilizers rules out an organic system in which fertility inputs feed organisms in the soil, which release substances that feed plants. Chemical fertilizers also contribute to climate change because they are produced with fossil fuels and limit the ability of soil to draw down (sequester) atmospheric carbon, thus undermining efforts to mitigate the climate em

## Hybrid Search (Vector + BM25 with RRF)

Combine vector similarity and keyword matching using Reciprocal Rank Fusion for best results.

In [28]:
def hybrid_search(query: str, limit: int = 10, agency: str | None = None) -> pd.DataFrame:
    """Hybrid search combining vector ANN + BM25 full-text, reranked with RRF."""
    embedding = model.encode(query, normalize_embeddings=True)
    results = (
        table.search(query_type="hybrid")
        .vector(embedding)
        .text(query)
        .metric("cosine")
        .limit(limit)
    )
    if agency:
        results = results.where(f"agency_code = '{agency}'")
    return results.to_pandas().drop(columns=["vector"])

In [29]:
hybrid_search("endangered species habitat protection")

,id,text,docket_id,agency_code,title,comment,posted_date,_relevance_score
0,FWS-HQ-ES-2025-0034-238053,"Comment from Vecchio, Deborah As one of 250,000 In Defense of Animals supporters and someone who cares deeply about wild animals, I urge you not to go through with this rule.<br/><br/>The Endangered Species Act is one of the world&rsquo;s most powerful legal tools for protecting species from extinction and has an impressive success rate of 99%. If this rule goes through it would essentially gut the ESA and cause massive harm to the species it protects. Most certainly, species would go extinct because of this change. <br/><br/>Often, endangered and threatened species are endangered or threatened because of a threat to their habitat. A species cannot survive without a home. Science shows that habitat loss is a leading cause of species extinction. When members of a species lose habitat, they lose shelter, breeding grounds, and access to food. To propose that the destruction of habitat does not harm a species goes against science and common sense. Because of this definition change, the crucial habitats that endangered animals need to survive could be opened up to logging, mining, development, and other activities that would threaten already endangered species. To save a species, the habitat the animal relies upon must be protected. <br/><br/>It is important to keep the term &ldquo;harm&rdquo; defined as it is so that the ESA can continue to have a 99% success rate of saving species. <br/><br/>Thank you for considering this comment.<br/><br/>Sincerely,<br/>Mrs. Deborah Vecchio<br/>Gilbertsville, PA 19525-0684",FWS-HQ-ES-2025-0034,FWS,"Comment from Vecchio, Deborah","As one of 250,000 In Defense of Animals supporters and someone who cares deeply about wild animals, I urge you not to go through with this rule.<br/><br/>The Endangered Species Act is one of the world&rsquo;s most powerful legal tools for protecting species from extinction and has an impressive success rate of 99%. If this rule goes through it would essentially gut the ESA and cause massive harm to the species it protects. Most certainly, species would go extinct because of this change. <br/><br/>Often, endangered and threatened species are endangered or threatened because of a threat to their habitat. A species cannot survive without a home. Science shows that habitat loss is a leading cause of species extinction. When members of a species lose habitat, they lose shelter, breeding grounds, and access to food. To propose that the destruction of habitat does not harm a species goes against science and common sense. Because of this definition change, the crucial habitats that endangered animals need to survive could be opened up to logging, mining, development, and other activities that would threaten already endangered species. To save a species, the habitat the animal relies upon must be protected. <br/><br/>It is important to keep the term &ldquo;harm&rdquo; defined as it is so that the ESA can continue to have a 99% success rate of saving species. <br/><br/>Thank you for considering this comment.<br/><br/>Sincerely,<br/>Mrs. Deborah Vecchio<br/>Gilbertsville, PA 19525-0684",2025-05-23T04:00:00Z,0.031754
1,FWS-HQ-ES-2025-0034-153049,"Comment from Redolfi, Luciana As one of 250,000 In Defense of Animals supporters and someone who cares deeply about wild animals, I urge you not to go through with this rule.<br/><br/>The Endangered Species Act is one of the world&rsquo;s most powerful legal tools for protecting species from extinction and has an impressive success rate of 99%. If this rule goes through it would essentially gut the ESA and cause massive harm to the species it protects. Most certainly, species would go extinct because of this change. <br/><br/>Often, endangered and threatened species are endangered or threatened because of a threat to their habitat. A species cannot survive without a home. Science shows that habitat loss is a leading cause of species exti

## Find Similar Comments

Given a comment ID, find other comments with similar content.

In [23]:
def find_similar(comment_id: str, limit: int = 10) -> pd.DataFrame:
    """Find comments similar to a given comment by its ID."""
    source = table.search().where(f"id = '{comment_id}'").limit(1).to_pandas()
    if source.empty:
        raise ValueError(f"Comment {comment_id} not found")

    vector = source.iloc[0]["vector"]
    return (
        table.search(vector)
        .metric("cosine")
        .where(f"id != '{comment_id}'")
        .limit(limit)
        .to_pandas()
        .drop(columns=["vector"])
    )

In [24]:
# Pick a comment from the vector search results and find similar ones
results = vector_search("climate change carbon emissions", limit=1)
first_id = results.iloc[0]["id"]
print(f"Finding comments similar to: {first_id}")
print(f"Text: {results.iloc[0]['text'][:200]}...\n")

find_similar(first_id)

Finding comments similar to: TREAS-DO-2021-0014-0071
Text: Comment from Vander Poel, James FIO Director Seitz,<br/><br/>Thank you, Federal Insurance Office, for taking up the issue of climate change and insurance. <br/><br/>Right now, insurance companies are ...



,id,text,docket_id,agency_code,title,comment,posted_date,_distance
0,BOEM-2022-0031-250652,"Comment from Fritzke, Johanna Dear Secretary Debra Haaland,<br/><br/>I&rsquo;m writing to share my concerns about BOEM&rsquo;s draft Five-Year Outer Continental Shelf Leasing Plan. I&rsquo;m distressed to see that the Interior Department has plans to hold up to 11 new lease sales: 10 in the Gulf of Mexico and one in Alaska&rsquo;s Cook Inlet. <br/><br/>It is more crucial than ever that the Biden administration uses the full power of the Executive branch, including your agency, to help avoid the worst impacts of the climate crisis.<br/><br/>New federal fossil fuel leasing is incompatible with President Biden&rsquo;s own climate and environmental justice commitments. We cannot meet our climate goals and open new fossil fuel extraction projects at the same time. If we wish to preserve a safe and livable climate for our families and communities, President Biden&rsquo;s Interior Department must swiftly bring an end to new federal offshore leasing.<br/><br/>Additionally, offshore drilling poses huge risks to the health of communities, workers, and wildlife, particularly in frontline and coastal communities who already bear the brunt of the climate crisis. This administration must swiftly usher a just transition to a clean energy future, not new federal fossil fuel leasing. <br/><br/>I sincerely urge you to recognize the harm that new federal leases pose, and follow through on your agency&rsquo;s own climate and environmental justice commitments. The time is now to use the powers of the executive branch to tackle the climate crisis, and offer no new leases in BOEM&rsquo;s final five-year program to protect our communities and our future.<br/><br/>Thank you for your time and attention to this important matter.<br/><br/>Sincerely,<br/>Johanna Fritzke<br/>Stacy, MN 55079",BOEM-2022-0031,BOEM,"Comment from Fritzke, Johanna","Dear Secretary Debra Haaland,<br/><br/>I&rsquo;m writing to share my concerns about BOEM&rsquo;s draft Five-Year Outer Continental Shelf Leasing Plan. I&rsquo;m distressed to see that the Interior Department has plans to hold up to 11 new lease sales: 10 in the Gulf of Mexico and one in Alaska&rsquo;s Cook Inlet. <br/><br/>It is more crucial than ever that the Biden administration uses the full power of the Executive branch, including your agency, to help avoid the worst impacts of the climate crisis.<br/><br/>New federal fossil fuel leasing is incompatible with President Biden&rsquo;s own climate and environmental justice commitments. We cannot meet our climate goals and open new fossil fuel extraction projects at the same time. If we wish to preserve a safe and livable climate for our families and communities, President Biden&rsquo;s Interior Department must swiftly bring an end to new federal offshore leasing.<br/><br/>Additionally, offshore drilling poses huge risks to the health of communities, workers, and wildlife, particularly in frontline and coastal communities who already bear the brunt of the climate crisis. This administration must swiftly usher a just transition to a clean energy future, not new federal fossil fuel leasing. <br/><br/>I sincerely urge you to recognize the harm that new federal leases pose, and follow through on your agency&rsquo;s own climate and environmental justice commitments. The time is now to use the powers of the executive branch to tackle the climate crisis, and offer no new leases in BOEM&rsquo;s final five-year program to protect our communities and our future.<br/><br/>Thank you for your time and attention to this important matter.<br/><br/>Sincerely,<br/>Johanna Fritzke<br/>Stacy, MN 55079",2023-03-18T04:00:00Z,0.368157
1,FAR-2021-0016-11972,"Comment on FR Doc # 2021-22266 The Biden Administration,<br/><br/>I&rsquo;m writing to urge the Federal Government to be expansive in its rule minimizing the risk of climate change in federal acquisitions. Climate experts say that we must reduce 

## Compare Search Methods

See how vector, text, and hybrid search return different results for the same query.

In [30]:
query = "offshore drilling environmental impact"

vec = set(vector_search(query, limit=20)["id"])
txt = set(text_search(query, limit=20)["id"])
hyb = set(hybrid_search(query, limit=20)["id"])

print(f"Query: '{query}'")
print(f"Vector-only results:  {len(vec)}")
print(f"Text-only results:   {len(txt)}")
print(f"Hybrid results:      {len(hyb)}")
print(f"\nOverlap (vector & text): {len(vec & txt)}")
print(f"Unique to vector:        {len(vec - txt)}")
print(f"Unique to text:          {len(txt - vec)}")

Query: 'offshore drilling environmental impact'
Vector-only results:  20
Text-only results:   20
Hybrid results:      20

Overlap (vector & text): 7
Unique to vector:        13
Unique to text:          13
